# Getting Started with Neo4j for Spatial Analysis

Introduction...

## Launch Neo4j locally using Docker

### Create a persistent data volume

To get started, you'll need to create a persistent data volume that Docker can use to store your database's files.  Creating a Docker volume allows you to persist any data loaded into the database, even if you have to stop, start, or re-run the Docker container housing your database.  Run the following from the command line to create a Docker volume for you Neo4j database:

```bash
docker volume create neo4j_volume
```

To check that the volume was created successfully, run the following from your command line:
```bash
docker volume ls
```

You should see a volume called `neo4j_volume` listed in the results.

### Download database plugins

Neo4j has a number of community-supported plugins that provide useful utilities for loading data, running advanced graph algorithms, and managing spatial data.  Download the latest release of each of these plugins from their respective GitHub pages:

- Awesome Procedures for Neo4j (APOC): 
[[Download link](https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/neo4j-apoc-procedures)]
[[Documentation](https://neo4j-contrib.github.io/neo4j-apoc-procedures/)]

- Efficient Graph Algorithms for Neo4j: 
[[Download link](https://github.com/neo4j-contrib/neo4j-graph-algorithms/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/neo4j-graph-algorithms)]
[[Documentation](https://neo4j.com/docs/graph-algorithms/current/)]

- Neo4j Spatial: 
[[Download link](https://github.com/neo4j-contrib/spatial/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/spatial)]
[[User Guide](https://neo4j-contrib.github.io/spatial/)]


Each of these plugins downloads as a .jar file.  Take these .jar files and move them to a folder on your computer where you can find them easily.  For this example, we'll assume you're saving them to your user folder inside of a folder called 'neo4j_plugins':

/Users/your_username/neo4j_plugins

### Launch the Docker container

Now you're finally ready to launch Neo4j using Docker!  Run the following from your command line.  Be sure to replace the file path in brackets (<>) below with the actual path to your neo4j_plugins folder (and remove the brackets):

```bash
docker run -d \
--name neo4j \
-p 7474:7474 -p 7687:7687 \
--mount source=neo4j_volume,target=/data \
-v </Users/your_username/neo4j_plugins>:/plugins \
-e NEO4J_apoc_export_file_enabled=true \
-e NEO4J_apoc_import_file_enabled=true \
-e NEO4J_apoc_import_file_use__neo4j__config=true \
-e NEO4J_dbms_security_procedures_unrestricted=apoc.\\\*,algo.\\\*,spatial.\\\* \
-e NEO4J_dbms_security_allow__csv__import__from__file__urls=true \
neo4j:3.4
```

What does this command do?  Breaking it down, here's what each argument means:

- **-d** : runs the container in "detached" mode, so that it keeps running in the background and will not shut off if you close out of your console window
- **--name** : the name Docker will give to your container; if you don't specify a name here, Docker will give your container a randomly-generated name
- **-p** : these are port mappings, indicating that Docker should forward information going in and out of port 7474 from the Neo4j container to port 7474 on your local machine; the same goes for port 7687
- **--mount** : takes the persistent volume named `neo4j_volume` and mounts it into the /data folder inside of the container; this is where the database's data and settings will get stored
- **-e**: environment variables that get passed to the database's configuration file on startup; these variables are required to make sure plugins will run correctly
- At the very end of the command, you'll notice we list **neo4j:3.4** as the final argument.  This specifies the Docker image and version to run inside of the container, and will download and launch Neo4j version 3.4 (the most current version as of this writing).  If newer versions are available, you can specify `neo4j:latest` to get the most recent version of Neo4j.

To check that the container is running, execute the following command in the command line:

```docker container ls```

You should see something like this, indicating that the container is successfully running:

```
CONTAINER ID    IMAGE        COMMAND                CREATED         STATUS          PORTS
blahblahblah    neo4j:3.4    "/sbin/tini -g -- /d…" 9 seconds ago   Up 8 seconds    0.0.0.0:7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp
```

## Connect to the database

Neo4j has a web-based access interface that you can view using a web browser.  Launch your web browser of choice and navigate to:

```http://localhost:7474/browser```

If the container is successfully initialized, you should be able to see a login interface in your web browser.  It will prompt you for a username and password.  Enter ```neo4j``` as both your username and password:

![Neo4j browser launch page](img_neo4j/neo4j_browser_launch.png)

Then, reset the default admit password to a new password of your choice when prompted:

![Neo4j browser launch page password setup](img_neo4j/neo4j_browser_launch_2.png)

## Load data into the database

Now, it's time to load in some data!  


Data model



### Create indexes

Prior to loading in any data, let's start off by adding some indexes to the database.  Adding indexes is a critical step, because the presence of indexes can really help improve load time.  We also want to do our best not to load duplicate data when adding data to the database, and indexes can help with this.  When loading in data, we will ask Neo4j to check for any duplicates of tweets, users, places, or hashtags that may be present prior to loading in new records.  If it encounters a tweet that is already present in the database, Neo4j can skip over this tweet and avoid loading a redundant record.  In order to check for duplicates, however, Neo4j first needs to search through all of the existing records on each record load to see what's already present in the database.  This search and insert process goes _much_ more quickly if data is pre-sorted using indexes.  

_What happens if we don't add indexes before loading in the data?_  Here is an example of 40 files being loaded into Neo4j installed on a home server.  In this scenario, the first 20 files were loaded prior to the addition of indexes, and the last 20 files were loaded after the addition of indexes:  

![visualization of load time before and after adding indexes](img_neo4j/neo4j_load_time_indexes.png)

Some of the load time depicted above is due to network latency and processing effort required in the database, but much of the load time can be attributed to the relative efficiency or ineffeciency due to the presence or absence of indexes.  For the first 20 files, you can see that the load time increases at a linear rate, with each file taking consistently longer to load.  For the last 20 files, you can see that the load time immediately drops after the addition of the indexes and stays more consistently flat, regardless of how many additional files we load.

To create an index, run the following command in the web admin console for your Neo4j instance.  Copy the command into the console, and then click the "play" button to execute the command: 

```cypher
CREATE INDEX ON :Tweet(tweet_id)
```

![Neo4j create index call](img_neo4j/neo4j_create_index.png)

The above command created an index on tweet_id, which is the unique identifier we'll be using to load tweets and check for duplicates.  Next, create an index on user_id, place_id, and hashtag_id to function in a similar manner.  Execute these commands one by one in the Neo4j web admin console:

```cypher
CREATE INDEX ON :User(user_id)
CREATE INDEX ON :Place(place_id)
CREATE INDEX ON :Hashtag(hashtag_id)
```

Finally, add a spatial index for the centroids of the Twitter Places that are present in our tweets.  This will enable some basic spatial query functionality after the data is loaded:

```cypher
CREATE INDEX ON :Place(centroid)
```

To check that the indexes have been successfully created, run the following:
```cypher
CALL db.indexes
```

![Neo4j results for CALL db.indexes](img_neo4j/neo4j_indexes.png)

### Execute load scripts

In [18]:
import Clean_Load_Scripts as cleanNLoad
import secrets

In [19]:
data_folder = '/Users/linkalis/Desktop/twitter_data/data_small_5000_split/'
logs_folder = '/Users/linkalis/Desktop/twitter_data/data_small_5000_split/logs/'

In [20]:
# Initialize extractor
extractor = cleanNLoad.Extractor(data_folder, logs_folder)

In [21]:
while extractor.next_file_available():
    next_file_data, next_file_name = extractor.get_next_file() # read in the next file
    cleaner = cleanNLoad.Cleaner(next_file_data, next_file_name, logs_folder) # clean the data (fix bounding boxes, add centroids, etc.)
    cleaned_data = cleaner.clean_data() 
    loader = cleanNLoad.Loader(cleaned_data, next_file_name, logs_folder) # initialize the loader
    loader.get_connection("neo4j", secrets.neo4j_host, secrets.neo4j_port, secrets.neo4j_username, secrets.neo4j_pwd) # create a db connection
    loader.load_data() # load the data

Extractor: Next file is: 500M_unicode_splitan.txt
Extractor: Reading file: /Users/linkalis/Desktop/twitter_data/data_small_5000_split/500M_unicode_splitan.txt
Extractor: Read 5000 data rows.
Cleaner: Finished cleaning records.


AttributeError: 'Session' object has no attribute 'session'

## Query the data

### Basic queries

### Spatial queries

### Advanced queries